In [22]:
import pandas as pd
import numpy as np

In [23]:
data = pd.read_csv("dataset.csv")

In [24]:
data.head()

,email,category
0,"URL: http://www.newsisfree.com/click/-1,817167...",not-spam
1,"On Thu, 19 Sep 2002, Bill Stoddard wrote:\n\n-...",not-spam
2,Dan Kohn <dan@dankohn.com> writes:\n\n\n\n> Gu...,not-spam
3,wintermute wrote:\n\n>>Anyone know where in Ir...,not-spam
4,"I attended the same conference, and was impres...",not-spam


In [25]:
data.tail()

,email,category
3791,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
3792,<html>\n\n<HEAD> \n\n<TITLE>creditfix</TITLE> ...,spam
3793,Hello:\n\n\n\nAre you satisfied with your ISP?...,spam
3794,<html><HEAD><TITLE>creditfix</TITLE> \n\n</HEA...,spam
3795,<HTML><HEAD></HEAD><BODY><b><FONT SIZE=3D4>In ...,spam


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression

In [27]:
X = data.drop('category',axis=1)
y = data['category']

In [28]:
type(X)

pandas.core.frame.DataFrame

In [29]:
import re

In [30]:
def cleanText(text): 
    clean = re.compile("<.*?>")
    return re.sub(clean,'',text)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=42)

In [32]:
X_train_clean = X_train['email'].apply(cleanText)
X_test_clean = X_test['email'].apply(cleanText)

In [33]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=5000,
    ngram_range=(1,2)
)
scaler = StandardScaler(with_mean=False)
le = LabelEncoder()

In [34]:
X_train_transformed = tfidf.fit_transform(X_train_clean)
X_test_transformed = tfidf.transform(X_test_clean)

In [35]:
X_test_transformed.shape

(1253, 5000)

In [36]:
X_train_transformed.shape

(2543, 5000)

In [37]:
y_train = le.fit_transform(y_train)

In [38]:
y_train.shape

(2543,)

In [39]:
linearRegression = LinearRegression()

In [40]:
linearRegression.fit(X_train_transformed,y_train)

LinearRegression()

In [41]:
y_pred = linearRegression.predict(X_test_transformed)

In [42]:
from sklearn.metrics import accuracy_score

In [43]:
y_test = le.fit_transform(y_test)

In [44]:
y_pred

array([ 0.03362572,  0.87623607,  0.10496734, ...,  0.28342888,
       -0.03463231,  1.00000023])

In [45]:
y_pred_continuous = np.array(y_pred)
y_pred_binary = (y_pred_continuous>=0.5).astype(int)

In [46]:
acc = accuracy_score(y_test,y_pred_binary)

In [47]:
print(acc)

0.9353551476456504


In [48]:
KNN = KNeighborsClassifier()

In [49]:
KNN.fit(X_train_transformed,y_train)

KNeighborsClassifier()

In [50]:
y_pred_knn = KNN.predict(X_test_transformed)

In [51]:
y_pred_knn

array([0, 1, 1, ..., 1, 1, 1])

In [52]:
acc2 = accuracy_score(y_test,y_pred_knn)
print(acc2)

0.7454110135674381


In [53]:
randomForest = RandomForestClassifier()

In [54]:
randomForest.fit(X_train_transformed,y_train)

RandomForestClassifier()

In [55]:
y_pred_rf = randomForest.predict(X_test_transformed)

In [56]:
y_pred_rf

array([0, 1, 0, ..., 0, 0, 1])

In [57]:
acc3 = accuracy_score(y_test,y_pred_rf)
acc3

0.9736632083000798

In [64]:
a = {'Model Name':[],'Mean_Absolute_Error_MAE':[],'Adj_R_Square':[],'Root_Mean_Squared_Error_RMSE':[],'Mean_Absolute_Percentage_Error_MAPE': [],'Mean_Squared_Error_MSE':[],'Root_Mean_Squared_Log_Error_RMSLE':[],'R2_Score':[]}
Results=pd.DataFrame(a)
Results.head()

,Model Name,Mean_Absolute_Error_MAE,Adj_R_Square,Root_Mean_Squared_Error_RMSE,Mean_Absolute_Percentage_Error_MAPE,Mean_Squared_Error_MSE,Root_Mean_Squared_Log_Error_RMSLE,R2_Score


In [69]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Use Classification versions of these models
MM = [
    LogisticRegression(), 
    LinearRegression(),
    KNeighborsClassifier(), 
    RandomForestClassifier()
]

Results = pd.DataFrame() # Initialize the results table

for model in MM: 
    # 1. Train the model
    model.fit(X_train_transformed, y_train)
    y_pred = model.predict(X_test_transformed)
    
    # 2. Logic for Linear Regression (Continuous to Binary)
    if isinstance(model, LinearRegression):
        # Convert continuous numbers (like 0.7 or 0.2) to 1 or 0
        y_pred = (y_pred >= 0.5).astype(int)
        
    # 3. Calculate Classification Metrics (Now works for all)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f'Model: {model.__class__.__name__}')
    print(f'Accuracy: {round(acc, 4)}')
    print(f'F1 Score: {round(f1, 4)}')
    print('-' * 30)

    # 4. Store results
    new_row = pd.DataFrame({
        'Model Name': [model.__class__.__name__],
        'Accuracy': [acc],
        'Precision': [prec],
        'Recall': [rec],
        'F1_Score': [f1]
    })
    Results = pd.concat([Results, new_row], ignore_index=True)

Model: LogisticRegression
Accuracy: 0.9832
F1 Score: 0.9826
------------------------------
Model: LinearRegression
Accuracy: 0.9354
F1 Score: 0.9349
------------------------------
Model: KNeighborsClassifier
Accuracy: 0.7454
F1 Score: 0.7905
------------------------------
Model: RandomForestClassifier
Accuracy: 0.9713
F1 Score: 0.9705
------------------------------
